In [ ]:
from collections import deque

# Bản đồ Sokoban
grid = [
    "#######",
    "#.....#",
    "#.###.#",
    "#.B.GP#",
    "#.....#",
    "#######"
]

ROWS, COLS = len(grid), len(grid[0])

# Tìm vị trí người chơi, hộp và đích
player_start = None
boxes_start = set()
goals = set()

for r in range(ROWS):
    for c in range(COLS):
        ch = grid[r][c]
        if ch == 'P':
            player_start = (r, c)
        elif ch == 'B':
            boxes_start.add((r, c))
        elif ch == 'G':
            goals.add((r, c))

# 4 hướng: (delta_r, delta_c, tên hướng)
directions = [
    (-1, 0, 'Up'),
    (1, 0, 'Down'),
    (0, -1, 'Left'),
    (0, 1, 'Right')
]

def is_solved(boxes, goals):
    return boxes == goals

# BFS setup
visited = set()
start_state = (player_start, frozenset(boxes_start))
visited.add(start_state)
queue = deque()
queue.append((player_start, frozenset(boxes_start), 0))

# Lưu cha và hướng đi từ cha đến con
parent = {}
move_dir = {}  # map state → direction string
parent[start_state] = None
move_dir[start_state] = None

end_state = None

while queue:
    player, boxes, steps = queue.popleft()

    if is_solved(boxes, goals):
        end_state = (player, boxes)
        print("Giải thành công sau", steps, "bước.")
        break
    for dr, dc, dir_name in directions:
        pr, pc = player[0] + dr, player[1] + dc
        if not (0 <= pr < ROWS and 0 <= pc < COLS):
            continue
        if grid[pr][pc] == '#':
            continue
        if (pr, pc) in boxes:
            # Người muốn đẩy hộp
            br, bc = pr + dr, pc + dc
            if not (0 <= br < ROWS and 0 <= bc < COLS):
                continue
            if grid[br][bc] == '#' or (br, bc) in boxes:
                continue
            new_boxes = set(boxes)
            new_boxes.remove((pr, pc))
            new_boxes.add((br, bc))
            new_state = ((pr, pc), frozenset(new_boxes))
            if new_state not in visited:
                visited.add(new_state)
                queue.append(((pr, pc), frozenset(new_boxes), steps + 1))
                parent[new_state] = (player, boxes)
                move_dir[new_state] = dir_name
        else:
            # Người đi bình thường
            new_state = ((pr, pc), boxes)
            if new_state not in visited:
                visited.add(new_state)
                queue.append(((pr, pc), boxes, steps + 1))
                parent[new_state] = (player, boxes)
                move_dir[new_state] = dir_name
else:
    print("Không tìm thấy đường đẩy hộp đến đích.")


if end_state:
    directions_path = []
    state = end_state
    while state and move_dir[state] is not None:
        directions_path.append(move_dir[state])
        state = parent[state]
    directions_path.reverse()

    print("Đường đi:")
    print(" → ".join(directions_path))


Giải thành công sau 8 bước.
Đường đi:
Down → Left → Left → Left → Left → Up → Right → Right
